# 加载库函数

In [1]:
import numpy as np
import scipy.sparse as sp
import networkx as nx
import networkx.algorithms.isomorphism as iso
import itertools
import os.path
import subprocess
import torch.nn.functional as F
import pickle
import os
import scipy.sparse as sp
import torch
import json
import scipy.io as sio
from torch.nn import Embedding
#import scipy.sparse as spsparse
from torch.nn import Parameter
import torch
import torch.nn as nn
from scipy.sparse import csr_matrix
import torch.nn.functional as F
import argparse
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score

# 加载数据，数据处理

In [2]:
"""
Disclaimer: functions defined from lines 15 to 36 in this file come from 
tkipf/gae original repository on Graph Autoencoders. Moreover, the
mask_test_edges_general_link_prediction function is borrowed from 
philipjackson's mask_test_edges pull request on this same repository.
"""

def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

# 任务二

In [4]:
def mask_test_edges_biased_negative_samples(adj, test_percent=10.):
    """
    Task 2: General Biased Negative Samples (B.N.S.) Directed Link
    Prediction: get Train and Test sets

    :param adj: complete sparse adjacency matrix of the graph
    :param test_percent: percentage of edges in test set
    :return: train incomplete adjacency matrix and test sets
    """

    # Remove diagonal elements of adjacency matrix
    adj = adj - sp.dia_matrix((adj.diagonal()[None, :], [0]), shape = adj.shape)
    adj.eliminate_zeros()
    val_edges, val_edges_false, test_edges, test_edges_false = None, None, None, None

    ## Retrieve all unidirectional edges
    adj_sym = (adj + adj.T).sign()
    adj_tilde = (adj_sym - adj).T
    adj_tilde.eliminate_zeros()
    edges_positive, _, _ = sparse_to_tuple(adj_tilde)

    # Number of positive (= to number of negative) test node pairs to sample
    num_test = int(np.floor(edges_positive.shape[0] / (100. / test_percent)))

    # Sampling of positive node pairs
    edges_idx = np.arange(edges_positive.shape[0])
    np.random.shuffle(edges_idx)
    test_edges_idx = edges_idx[:num_test]
    test_edges = edges_positive[test_edges_idx]

    # In this setting, the reverse node pairs constitute negative samples
    test_edges_false = np.fliplr(test_edges)

    # Get training incomplete adjacency matrix
    train_edges = np.delete(edges_positive, np.hstack([test_edges_idx]), axis = 0)
    #data = np.ones(train_edges.shape[0])
    #adj_train = sp.csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape = adj.shape)

    # Validation set: not implemented for Task 2
    # therefore, val_edges and val_edges_false are None
    return train_edges, val_edges, val_edges_false, test_edges, test_edges_false

In [6]:
def load_data(dataset):
    """ Load datasets from text files

    :param dataset: 'cora', 'citeseer' or 'google' graph dataset.
    :return: n*n sparse adjacency matrix and n*f node-level feature matrix

    Note: in this paper, all three datasets are assumed to be featureless.
    As a consequence, the feature matrix is the identity matrix I_n.
    """
    if dataset == 'cora':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/cora.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Cora raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset == 'citeseer':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/citeseer.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Citeseer raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset == 'google':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/GoogleNw.txt",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        features = sp.identity(adj.shape[0])

    else:
        raise ValueError('Undefined dataset!')

    return adj, features

In [7]:
def get_graph(DATASET='cora', task=1):
    adj_init, features = load_data(DATASET)
    if task==1:
        train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_general_link_prediction(adj_init, 10, 5)
    elif task==2:
        train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_biased_negative_samples(adj_init)
    else:
        train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_bidirectional_link_prediction(adj_init)
    
    all_negative_edges = np.argwhere(adj_init.toarray()==0)
    
    if task==2 or task==3:
        all_train_edges_false = np.delete(all_negative_edges, test_edges_false, axis = 0)
    else:
        all_train_edges_false = np.delete(all_negative_edges, np.vstack([test_edges_false, val_edges_false]), axis = 0)
    
    if DATASET=='cora':
        nodes_number = 2708
    elif DATASET=='citeseer':
        nodes_number = 3327
    
    ################## 处理测试集  #############################
    test_posi_mask = [test_edges[i][0]*nodes_number+test_edges[i][1] for i in range(len(test_edges))]
    test_false_mask = [test_edges_false[i][0]*nodes_number+test_edges_false[i][1] for i in range(len(test_edges_false))]
    test_mask = test_posi_mask + test_false_mask
    test_mask = torch.tensor(test_mask)
    test_edge_true_or_false = torch.zeros(len(test_mask))
    test_edge_true_or_false[:len(test_posi_mask)] = 1
    
    ############### 处理训练集  ################################
    negative_index = np.random.choice(range(all_train_edges_false.shape[0]), train_edges.shape[0], replace=False)
    train_edges_false = all_train_edges_false[negative_index]
    train_posi_mask = [train_edges[i][0]*nodes_number+train_edges[i][1] for i in range(len(train_edges))]
    train_false_mask = [train_edges_false[i][0]*nodes_number+train_edges_false[i][1] for i in range(len(train_edges_false))]
    train_mask = train_posi_mask + train_false_mask
    train_mask = torch.tensor(train_mask)
    #train_mask = torch.tensor(train_posi_mask)
    train_edge_true_or_false = torch.zeros(len(train_mask))
    train_edge_true_or_false[:len(train_posi_mask)] = 1
    
    return train_edge_true_or_false, test_edge_true_or_false, train_mask.long(), test_mask.long(), nodes_number

# 定义模型

In [9]:
def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    ap = average_precision_score(true_edges, predict_edges)
    auc = roc_auc_score(true_edges, predict_edges)
    print("AP SCORE： ", ap)
    print("AUC SCORE: ", auc)
    return ap, auc

################  更新节点内容  #######################################
class reconstruction_graph(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, context_embedding):
        super(reconstruction_graph, self).__init__()
        self.context_embedding = Parameter(context_embedding)
        
    def forward(self, importance_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(self.context_embedding[:, None]-self.context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((self.context_embedding.unsqueeze(1) - importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        #return 2*sigmoid_modle(nodes_similar)-1
        return torch.exp(-nodes_similar)
        
    
class update_nodes_embedding(nn.Module):
    def __init__(self, context_embedding):
        super(update_nodes_embedding, self).__init__()
        self.reconstruction_module = reconstruction_graph(context_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, importance_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(importance_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.context_embedding.detach(), ap, auc

####################  更新节点重要性  ######################################
class reconstruction_graph_importance(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, importance_embedding):
        super(reconstruction_graph_importance, self).__init__()
        self.importance_embedding = Parameter(importance_embedding)
        
    def forward(self, context_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(context_embedding[:, None]-context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((context_embedding.unsqueeze(1) - self.importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        #return 2*sigmoid_modle(nodes_similar)-1
        return torch.exp(-nodes_similar)

class update_nodes_embedding_importance(nn.Module):
    def __init__(self, importance_embedding):
        super(update_nodes_embedding_importance, self).__init__()
        self.reconstruction_module = reconstruction_graph_importance(importance_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, context_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(context_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.importance_embedding.detach(), ap, auc

In [10]:
context_embedding_dim = 12
importance_embedding_dim = 48
beta = 5.0
ALPHA = 1.0
LR = 0.01
MOMENTUM = 0.9
EPOCH = 5
N = 25.0    # 目前25最佳
sigmoid_modle = torch.nn.Sigmoid()

dataset = 'citeseer'

In [11]:
train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(DATASET=dataset, task=2)
context_embedding = Embedding(nodes_number, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
nodes_context_embeddings = context_embedding.weight.detach()

importance_embedding = Embedding(importance_embedding_dim, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
nodes_importance_embeddings = importance_embedding.weight.detach()
    
auc_first_best_auc = 0
auc_first_best_ap = 0
ap_first_best_auc = 0
ap_first_best_ap = 0

for module_epoch in range(40):
    print("######################### 模块循环 ： %d ##########################"%module_epoch)
    update_nodes_module = update_nodes_embedding(nodes_context_embeddings)
    nodes_context_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_importance_embeddings)
    if auc > auc_first_best_auc:
        auc_first_best_auc = auc
        auc_first_best_ap = ap
    if ap > ap_first_best_ap:
        ap_first_best_auc = auc
        ap_first_best_ap = ap
    update_nodes_importance_module = update_nodes_embedding_importance(nodes_importance_embeddings)
    nodes_importance_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_context_embeddings)
    if auc > auc_first_best_auc:
        auc_first_best_auc = auc
        auc_first_best_ap = ap
    if ap > ap_first_best_ap:
        ap_first_best_auc = auc
        ap_first_best_ap = ap

######################### 模块循环 ： 0 ##########################
Epoch: 00, Loss: 2151.0381
Epoch: 01, Loss: 2145.3228
Epoch: 02, Loss: 2134.5908
Epoch: 03, Loss: 2119.6936
Epoch: 04, Loss: 2101.6619
AP SCORE：  0.5278963687010906
AUC SCORE:  0.5401213287632501
Epoch: 00, Loss: 2068.1709
Epoch: 01, Loss: 2049.1282
Epoch: 02, Loss: 2029.6368
Epoch: 03, Loss: 2009.9836
Epoch: 04, Loss: 1990.3461
AP SCORE：  0.5953386311154727
AUC SCORE:  0.5894464809202336
######################### 模块循环 ： 1 ##########################
Epoch: 00, Loss: 1970.8458
Epoch: 01, Loss: 1968.3511
Epoch: 02, Loss: 1963.6436
Epoch: 03, Loss: 1957.0024
Epoch: 04, Loss: 1948.6991
AP SCORE：  0.6152460582995793
AUC SCORE:  0.6069563147008199
Epoch: 00, Loss: 1938.9890
Epoch: 01, Loss: 1928.1100
Epoch: 02, Loss: 1916.2781
Epoch: 03, Loss: 1903.6858
Epoch: 04, Loss: 1890.5012
AP SCORE：  0.64243808044471
AUC SCORE:  0.6306317925010293
######################### 模块循环 ： 2 ##########################
Epoch: 00, Loss: 1876.8690
Epoch

Epoch: 00, Loss: 1204.1414
Epoch: 01, Loss: 1203.5198
Epoch: 02, Loss: 1202.3400
Epoch: 03, Loss: 1200.6597
Epoch: 04, Loss: 1198.5315
AP SCORE：  0.7919347017766972
AUC SCORE:  0.7628483985694515
Epoch: 00, Loss: 1196.0034
Epoch: 01, Loss: 1193.1194
Epoch: 02, Loss: 1189.9188
Epoch: 03, Loss: 1186.4382
Epoch: 04, Loss: 1182.7102
AP SCORE：  0.7935812098224685
AUC SCORE:  0.7641777570547765
######################### 模块循环 ： 19 ##########################
Epoch: 00, Loss: 1178.7646
Epoch: 01, Loss: 1178.1685
Epoch: 02, Loss: 1177.0364
Epoch: 03, Loss: 1175.4238
Epoch: 04, Loss: 1173.3817
AP SCORE：  0.7943319523859202
AUC SCORE:  0.7648325157117276
Epoch: 00, Loss: 1170.9558
Epoch: 01, Loss: 1168.1881
Epoch: 02, Loss: 1165.1168
Epoch: 03, Loss: 1161.7765
Epoch: 04, Loss: 1158.1989
AP SCORE：  0.7958361768763065
AUC SCORE:  0.7662710006398779
######################### 模块循环 ： 20 ##########################
Epoch: 00, Loss: 1154.4121
Epoch: 01, Loss: 1153.8397
Epoch: 02, Loss: 1152.7531
Epoch: 03

Epoch: 04, Loss: 874.1839
AP SCORE：  0.8165693547588824
AUC SCORE:  0.790090326932902
Epoch: 00, Loss: 873.0221
Epoch: 01, Loss: 871.6969
Epoch: 02, Loss: 870.2272
Epoch: 03, Loss: 868.6294
Epoch: 04, Loss: 866.9192
AP SCORE：  0.8168419887898626
AUC SCORE:  0.7904226665542333
######################### 模块循环 ： 37 ##########################
Epoch: 00, Loss: 865.1100
Epoch: 01, Loss: 864.8367
Epoch: 02, Loss: 864.3180
Epoch: 03, Loss: 863.5796
Epoch: 04, Loss: 862.6445
AP SCORE：  0.81706633237974
AUC SCORE:  0.7907252444184305
Epoch: 00, Loss: 861.5339
Epoch: 01, Loss: 860.2676
Epoch: 02, Loss: 858.8627
Epoch: 03, Loss: 857.3354
Epoch: 04, Loss: 855.7005
AP SCORE：  0.8175425975560254
AUC SCORE:  0.7913750427825259
######################### 模块循环 ： 38 ##########################
Epoch: 00, Loss: 853.9707
Epoch: 01, Loss: 853.7094
Epoch: 02, Loss: 853.2134
Epoch: 03, Loss: 852.5071
Epoch: 04, Loss: 851.6130
AP SCORE：  0.8176815604606913
AUC SCORE:  0.7917073824038572
Epoch: 00, Loss: 850.5508


In [14]:
for module_epoch in range(20):
    print("######################### 模块循环 ： %d ##########################"%module_epoch)
    update_nodes_module = update_nodes_embedding(nodes_context_embeddings)
    nodes_context_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_importance_embeddings)
    if auc > auc_first_best_auc:
        auc_first_best_auc = auc
        auc_first_best_ap = ap
    if ap > ap_first_best_ap:
        ap_first_best_auc = auc
        ap_first_best_ap = ap
    update_nodes_importance_module = update_nodes_embedding_importance(nodes_importance_embeddings)
    nodes_importance_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_context_embeddings)
    if auc > auc_first_best_auc:
        auc_first_best_auc = auc
        auc_first_best_ap = ap
    if ap > ap_first_best_ap:
        ap_first_best_auc = auc
        ap_first_best_ap = ap

######################### 模块循环 ： 0 ##########################
Epoch: 00, Loss: 833.1155
Epoch: 01, Loss: 832.8760
Epoch: 02, Loss: 832.4214
Epoch: 03, Loss: 831.7739
Epoch: 04, Loss: 830.9542
AP SCORE：  0.8185433929309507
AUC SCORE:  0.7929772173749138
Epoch: 00, Loss: 829.9806
Epoch: 01, Loss: 828.8698
Epoch: 02, Loss: 827.6376
Epoch: 03, Loss: 826.2977
Epoch: 04, Loss: 824.8629
AP SCORE：  0.8186981470918251
AUC SCORE:  0.7934286040247817
######################### 模块循环 ： 1 ##########################
Epoch: 00, Loss: 823.3447
Epoch: 01, Loss: 823.1152
Epoch: 02, Loss: 822.6795
Epoch: 03, Loss: 822.0592
Epoch: 04, Loss: 821.2733
AP SCORE：  0.8187468000898113
AUC SCORE:  0.7936022142747308
Epoch: 00, Loss: 820.3401
Epoch: 01, Loss: 819.2756
Epoch: 02, Loss: 818.0945
Epoch: 03, Loss: 816.8101
Epoch: 04, Loss: 815.4346
AP SCORE：  0.81902153219393
AUC SCORE:  0.7939742362389076
######################### 模块循环 ： 2 ##########################
Epoch: 00, Loss: 813.9791
Epoch: 01, Loss: 813.7589


Epoch: 00, Loss: 702.3466
Epoch: 01, Loss: 701.7743
Epoch: 02, Loss: 701.1388
Epoch: 03, Loss: 700.4471
Epoch: 04, Loss: 699.7056
AP SCORE：  0.8224131656660285
AUC SCORE:  0.7999513891300142
######################### 模块循环 ： 19 ##########################
Epoch: 00, Loss: 698.9200
Epoch: 01, Loss: 698.8009
Epoch: 02, Loss: 698.5746
Epoch: 03, Loss: 698.2523
Epoch: 04, Loss: 697.8440
AP SCORE：  0.8224874141276679
AUC SCORE:  0.8000853170371178
Epoch: 00, Loss: 697.3585
Epoch: 01, Loss: 696.8046
Epoch: 02, Loss: 696.1891
Epoch: 03, Loss: 695.5194
Epoch: 04, Loss: 694.8013
AP SCORE：  0.8226226679067618
AUC SCORE:  0.8002192449442215


In [15]:
for module_epoch in range(40):
    print("######################### 模块循环 ： %d ##########################"%module_epoch)
    update_nodes_module = update_nodes_embedding(nodes_context_embeddings)
    nodes_context_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_importance_embeddings)
    if auc > auc_first_best_auc:
        auc_first_best_auc = auc
        auc_first_best_ap = ap
    if ap > ap_first_best_ap:
        ap_first_best_auc = auc
        ap_first_best_ap = ap
    update_nodes_importance_module = update_nodes_embedding_importance(nodes_importance_embeddings)
    nodes_importance_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_context_embeddings)
    if auc > auc_first_best_auc:
        auc_first_best_auc = auc
        auc_first_best_ap = ap
    if ap > ap_first_best_ap:
        ap_first_best_auc = auc
        ap_first_best_ap = ap

######################### 模块循环 ： 0 ##########################
Epoch: 00, Loss: 694.0406
Epoch: 01, Loss: 693.9250
Epoch: 02, Loss: 693.7059
Epoch: 03, Loss: 693.3937
Epoch: 04, Loss: 692.9981
AP SCORE：  0.8224762369066543
AUC SCORE:  0.8002043640656544
Epoch: 00, Loss: 692.5278
Epoch: 01, Loss: 691.9910
Epoch: 02, Loss: 691.3947
Epoch: 03, Loss: 690.7458
Epoch: 04, Loss: 690.0499
AP SCORE：  0.8227064035286992
AUC SCORE:  0.8004622992941504
######################### 模块循环 ： 1 ##########################
Epoch: 00, Loss: 689.3127
Epoch: 01, Loss: 689.2008
Epoch: 02, Loss: 688.9883
Epoch: 03, Loss: 688.6855
Epoch: 04, Loss: 688.3019
AP SCORE：  0.8228187847673047
AUC SCORE:  0.8005714257369755
Epoch: 00, Loss: 687.8461
Epoch: 01, Loss: 687.3256
Epoch: 02, Loss: 686.7474
Epoch: 03, Loss: 686.1182
Epoch: 04, Loss: 685.4434
AP SCORE：  0.8229205977980347
AUC SCORE:  0.8007301551083574
######################### 模块循环 ： 2 ##########################
Epoch: 00, Loss: 684.7284
Epoch: 01, Loss: 684.619

Epoch: 00, Loss: 625.8694
Epoch: 01, Loss: 625.5448
Epoch: 02, Loss: 625.1840
Epoch: 03, Loss: 624.7910
Epoch: 04, Loss: 624.3695
AP SCORE：  0.8243952435130477
AUC SCORE:  0.8034087132504304
######################### 模块循环 ： 19 ##########################
Epoch: 00, Loss: 623.9225
Epoch: 01, Loss: 623.8544
Epoch: 02, Loss: 623.7253
Epoch: 03, Loss: 623.5412
Epoch: 04, Loss: 623.3077
AP SCORE：  0.8244509355680335
AUC SCORE:  0.8035128794003997
Epoch: 00, Loss: 623.0303
Epoch: 01, Loss: 622.7134
Epoch: 02, Loss: 622.3612
Epoch: 03, Loss: 621.9775
Epoch: 04, Loss: 621.5658
AP SCORE：  0.8245178462982676
AUC SCORE:  0.8036170455503693
######################### 模块循环 ： 20 ##########################
Epoch: 00, Loss: 621.1293
Epoch: 01, Loss: 621.0629
Epoch: 02, Loss: 620.9367
Epoch: 03, Loss: 620.7568
Epoch: 04, Loss: 620.5290
AP SCORE：  0.8245522889760102
AUC SCORE:  0.8036616881860705
Epoch: 00, Loss: 620.2579
Epoch: 01, Loss: 619.9484
Epoch: 02, Loss: 619.6043
Epoch: 03, Loss: 619.2295
Epoch:

Epoch: 00, Loss: 581.7847
Epoch: 01, Loss: 581.7377
Epoch: 02, Loss: 581.6485
Epoch: 03, Loss: 581.5214
Epoch: 04, Loss: 581.3602
AP SCORE：  0.8240857497827462
AUC SCORE:  0.8036964102360604
Epoch: 00, Loss: 581.1684
Epoch: 01, Loss: 580.9492
Epoch: 02, Loss: 580.7057
Epoch: 03, Loss: 580.4401
Epoch: 04, Loss: 580.1552
AP SCORE：  0.8241365264984357
AUC SCORE:  0.803701370528916
######################### 模块循环 ： 38 ##########################
Epoch: 00, Loss: 579.8527
Epoch: 01, Loss: 579.8065
Epoch: 02, Loss: 579.7189
Epoch: 03, Loss: 579.5940
Epoch: 04, Loss: 579.4356
AP SCORE：  0.8241214058340566
AUC SCORE:  0.8037063308217718
Epoch: 00, Loss: 579.2472
Epoch: 01, Loss: 579.0319
Epoch: 02, Loss: 578.7927
Epoch: 03, Loss: 578.5318
Epoch: 04, Loss: 578.2518
AP SCORE：  0.8240232318669711
AUC SCORE:  0.8036616881860705
######################### 模块循环 ： 39 ##########################
Epoch: 00, Loss: 577.9546
Epoch: 01, Loss: 577.9092
Epoch: 02, Loss: 577.8232
Epoch: 03, Loss: 577.7004
Epoch: 

In [17]:
print("AUC FIRST BEST AUC: ", auc_first_best_auc)
print("AUC FIRST BEST AP: ", auc_first_best_ap)
print("AP FIRST BEST AUC: ", ap_first_best_auc)
print("AP FIRST BEST AP: ", ap_first_best_ap)

AUC FIRST BEST AUC:  0.8037708146288958
AUC FIRST BEST AP:  0.8240613871984364
AP FIRST BEST AUC:  0.80376585433604
AP FIRST BEST AP:  0.8246125682962402
